# Time Series Analysis

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from statsmodels.tsa.statespace.sarimax import SARIMAX, SARIMAXResults
from statsmodels.tsa.stattools import adfuller  
from sklearn.model_selection import train_test_split

import warnings
warnings.simplefilter(action="ignore")

In [ ]:
def model(df, target):
    train, test = train_test_split(df[target], train_size = .65, shuffle = False)
    
    sarimax = Sarimax(endog = train, seasonal_order)